# Import Libraries                            


In [1]:
import kagglehub
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
import torchvision.transforms as trans
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

#Download dataset

In [2]:
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")
print("Path to dataset: ", path)

Using Colab cache for faster access to the 'microsoft-catsvsdogs-dataset' dataset.
Path to dataset:  /kaggle/input/microsoft-catsvsdogs-dataset


#Declare parameters

In [3]:
num_classes = 2
batch_size = 32
learning_rate = 0.0005
num_epochs = 40

#Create CNN

In [4]:
class CNN(nn.Module):
    def __init__(self, in_channels = 3, num_classes = 2):
        super(CNN, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.linear_dropout = nn.Dropout(p=0.3) # Avoid overfitting
        self.convd_dropout = nn.Dropout2d(p=0.3)
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=3, stride=1, padding=1) # Convolutional layer => Features extracting
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.bn2 = nn.BatchNorm2d(num_features=128)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2) # Reduce image size
        self.gap = nn.AdaptiveAvgPool2d(1) # Flatten inputsize 4D->1D
        self.fc = nn.Linear(in_features=256, out_features=num_classes) # Features->Logits

    def forward(self, x):
        # Features extracting
        # Conv -> BN -> ReLU -> Pool
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.maxpool(self.relu(self.bn2(self.conv2(x))))
        x = self.maxpool(self.relu(self.bn3(self.conv3(x))))
        x = self.convd_dropout(x)

        # Classify
        # GAP -> Flatten -> Dropout -> Fully connect
        # GAP: Global Average Pooling: Multidimension Features Matrix -> Vector Features 1D
        x = self.gap(x)
        x = torch.flatten(x, 1)
        x = self.linear_dropout(x)
        x = self.fc(x)

        return x
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Image transformation and augmentation for training

# Use imagenet mean and std to normalize image
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

# Transformer for training
transform_augment = trans.Compose([
    trans.RandomResizedCrop(128, scale=(0.7, 1.0)),
    trans.RandomHorizontalFlip(),
    trans.RandomRotation(10),
    trans.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    trans.ToTensor(),
    trans.Normalize(mean=imagenet_mean, std=imagenet_std) # Change value later to enhance accuracy
])

# Transformer for validating and testing
transform_normal = trans.Compose([
    trans.Resize(136),
    trans.CenterCrop(128),
    trans.ToTensor(),
    trans.Normalize(mean=imagenet_mean, std=imagenet_std)
])



# Split & Load dataset

In [6]:
#Data path
dataset = '/content/data'

#Create directories
data_splits = ['train', 'valid', 'test']
class_name = ['Cat', 'Dog']
for split in data_splits:
    for name in class_name:
        temp_path = os.path.join(dataset, split, name)
        os.makedirs(temp_path, exist_ok=True)
        print("Create ", temp_path)

#Split images into directories
for name in class_name:
    source = os.path.join(path, 'PetImages', name)
    list_file = [] # List containing readable images
    for file in os.listdir(source):
        file_path = os.path.join(source, file)
        # Ignore error images
        try:
            img = Image.open(file_path)
            img.verify()
            list_file.append(file) # Append readable images to list
        except (IOError, SyntaxError, Image.UnidentifiedImageError, Image.DecompressionBombError):
            print(f'Error file: {file_path}')


    #Split images
    train_file, test_file = train_test_split(list_file, test_size=0.2, random_state=22)
    test_file, valid_file = train_test_split(test_file, test_size=0.5, random_state=22)
    split = {'train': train_file, 'valid': valid_file, 'test': test_file}
    for directory, files in split.items():
        for file in files:
            shutil.copy2(os.path.join(source, file), os.path.join(dataset, directory, name))




Create  /content/data/train/Cat
Create  /content/data/train/Dog
Create  /content/data/valid/Cat
Create  /content/data/valid/Dog
Create  /content/data/test/Cat
Create  /content/data/test/Dog
Error file: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/Thumbs.db
Error file: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Error file: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg
Error file: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/Thumbs.db


In [7]:
#Create dataset
train_dataset = datasets.ImageFolder(os.path.join(dataset, 'train'), transform=transform_augment)
valid_dataset = datasets.ImageFolder(os.path.join(dataset, 'valid'), transform=transform_normal)
test_dataset = datasets.ImageFolder(os.path.join(dataset, 'test'), transform=transform_normal)
#Create Dataset loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Validate model during training on validation dataset

In [8]:
def validate(model, val_loader, criterion):
    model.eval() #Evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            pred = model(data)
            loss = criterion(pred, target)
            val_loss += loss.item()

    #Return average loss
    return val_loss/len(val_loader)


#Create and train model

In [9]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay= 0.0001, betas=[0.9, 0.995])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5) # Modify learning rate after an epoch

In [10]:
for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()   #Train model
    for batch_idx, (data, target) in enumerate(train_loader):   #Learn by each batch
        data = data.to(device=device)
        target = target.to(device=device)

        pred = model(data)
        loss = criterion(pred, target)
        total_loss += loss.item()
        avg_loss = total_loss / (batch_idx + 1)
        # Print loss after each 5 batches
        if batch_idx % 5 == 0:
          print(f'epoch: {epoch}, batch: {batch_idx} with loss: {loss.item()} and average loss:{avg_loss}')
          print(f"Epoch {epoch+1}: LR={optimizer.param_groups[0]['lr']:.6f}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    val_loss = validate(model, valid_loader, criterion)
    print(f'Validation loss: {val_loss}')
    scheduler.step(val_loss)

Streaming output truncated to the last 5000 lines.
epoch: 20, batch: 50 with loss: 0.3672405183315277 and average loss:0.45016369223594666
Epoch 21: LR=0.000500
epoch: 20, batch: 55 with loss: 0.4162135720252991 and average loss:0.4504016526043415
Epoch 21: LR=0.000500
epoch: 20, batch: 60 with loss: 0.5126811265945435 and average loss:0.45016572612230893
Epoch 21: LR=0.000500
epoch: 20, batch: 65 with loss: 0.4899403750896454 and average loss:0.4507821048751022
Epoch 21: LR=0.000500
epoch: 20, batch: 70 with loss: 0.521050751209259 and average loss:0.4526138318256593
Epoch 21: LR=0.000500
epoch: 20, batch: 75 with loss: 0.38083696365356445 and average loss:0.4533926154437818
Epoch 21: LR=0.000500
epoch: 20, batch: 80 with loss: 0.44468170404434204 and average loss:0.4531815743740694
Epoch 21: LR=0.000500
epoch: 20, batch: 85 with loss: 0.40363189578056335 and average loss:0.45430581133032955
Epoch 21: LR=0.000500
epoch: 20, batch: 90 with loss: 0.45076966285705566 and average loss:0.4

#Check accuracy

In [11]:
def check_accuracy(test_loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()   #evaluation mode

    with torch.no_grad():
        for x,y in test_loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)

            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 17395 / 19998 with accuracy 86.98
Got 2234 / 2500 with accuracy 89.36


In [ ]:
PATH = 'Cats_Dogs.pt'
torch.save(model.state_dict(), PATH)

In [13]:
cnn = CNN()
cnn.load_state_dict(torch.load(PATH)) # Get params from ram for model
cnn.to(device)
cnn.eval()
check_accuracy(test_loader, cnn)

Got 2234 / 2500 with accuracy 89.36


In [14]:
!zip -r /content/data.zip /content/data

Streaming output truncated to the last 5000 lines.
  adding: content/data/train/Dog/1932.jpg (deflated 3%)
  adding: content/data/train/Dog/2024.jpg (deflated 1%)
  adding: content/data/train/Dog/11058.jpg (deflated 4%)
  adding: content/data/train/Dog/7946.jpg (deflated 0%)
  adding: content/data/train/Dog/2705.jpg (deflated 8%)
  adding: content/data/train/Dog/10201.jpg (deflated 1%)
  adding: content/data/train/Dog/4125.jpg (deflated 1%)
  adding: content/data/train/Dog/10173.jpg (deflated 37%)
  adding: content/data/train/Dog/2242.jpg (deflated 1%)
  adding: content/data/train/Dog/7659.jpg (deflated 0%)
  adding: content/data/train/Dog/2682.jpg (deflated 1%)
  adding: content/data/train/Dog/5253.jpg (deflated 1%)
  adding: content/data/train/Dog/7977.jpg (deflated 27%)
  adding: content/data/train/Dog/4088.jpg (deflated 0%)
  adding: content/data/train/Dog/7415.jpg (deflated 0%)
  adding: content/data/train/Dog/9353.jpg (deflated 1%)
  adding: content/data/train/Dog/10043.jpg (defl